# Cities' capacity overload
---

In this notebook we calculate the estimate capacity overload day of all cities in Brazil.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-packages" data-toc-modified-id="Import-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import packages</a></span></li><li><span><a href="#Model-&amp;-simulation-parameters" data-toc-modified-id="Model-&amp;-simulation-parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model &amp; simulation parameters</a></span></li><li><span><a href="#Import-data" data-toc-modified-id="Import-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Import data</a></span></li><li><span><a href="#Calculate-dday" data-toc-modified-id="Calculate-dday-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Calculate dday</a></span><ul class="toc-item"><li><span><a href="#Errors" data-toc-modified-id="Errors-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Errors</a></span></li></ul></li></ul></div>

## Import packages

In [1]:
import sys
sys.path.append("..")
sys.path.append("../src")

In [2]:
%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH, THEMES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999

import warnings

warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Centering and fixing title
def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Adding custom colorscales (to add one: themes/custom_colorscales.yaml)
import yaml
custom_colorscales = yaml.load(open(THEMES_PATH / 'custom_colorscales.yaml', 'r'))
cf.colors._custom_scales['qual'].update(custom_colorscales)
cf.colors.reset_scales()

# Setting cuffilinks template (use it with .iplot(theme='custom')
cf.themes.THEMES['custom'] = yaml.load(open(THEMES_PATH / 'cufflinks_template.yaml', 'r'))

## Model & simulation parameters

In [3]:
config = yaml.load(open('../src/configs/config.yaml', 'r'))
config['br']['seir_parameters']

{'mild_duration': 6,
 'severe_duration': 6,
 'critical_duration': 8,
 'fatality_ratio': 0.02,
 'doubling_rate': 1.15,
 'incubation_period': 5,
 'i1_percentage': 0.855,
 'i2_percentage': 0.12,
 'i3_percentage': 0.025,
 'infected_health_care_proportion': 0.05}

In [4]:
config['simulator']

{'max_days': 90,
 'scenarios': {'worst': {'nothing': {'R0': 3.97},
   'isolation': {'R0': 1.31},
   'lockdown': {'R0': 0.37},
   'test_delay': 0},
  'best': {'nothing': {'R0': 3.74},
   'isolation': {'R0': 1.21},
   'lockdown': {'R0': 0.28},
   'test_delay': 0}}}

## Import data

In [5]:
from src.model import simulator
from src import loader

In [6]:
import math
from datetime import datetime
    
def read_data(country, config, refresh_rate):

    df = pd.read_csv(config[country]['data_endpoints']['raw'])
    df[[c for c in df.columns if 'last_updated' in c]] = df[[c for c in df.columns if 'last_updated' in c]].apply(pd.to_datetime)

    return  df

# health_infra = loader._download_from_drive(config['br']['drive_paths']['health_infrastructure'])
data = read_data('br', config, refresh_rate=10)

In [7]:
data

,country_iso,country_name,state_id,state_name,city_id,city_name,population,health_system_region,last_updated_number_ventilators,author_number_ventilators,number_ventilators,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_icu_beds,author_number_icu_beds,number_icu_beds,last_updated,confirmed_cases,deaths,state,daily_cases,infectious_period_cases,notification_rate,active_cases,data_last_refreshed
0,BR,Brasil,AC,Acre,1200013,Acrelândia,15256,Acre,2020-02-01,SUS,NaN,2020-02-01,SUS,3.2,2020-02-01,SUS,NaN,2020-04-26,12.0,0.0,AC,0.0,2.0,1.0,2.0,2020-04-27 18:41:03
1,BR,Brasil,AC,Acre,1200054,Assis Brasil,7417,Brasiléia,2020-02-01,SUS,NaN,2020-02-01,SUS,2.4,2020-02-01,SUS,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-27 18:41:03
2,BR,Brasil,AC,Acre,1200104,Brasiléia,26278,Brasiléia,2020-02-01,SUS,NaN,2020-02-01,SUS,8.0,2020-02-01,SUS,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-27 18:41:03
3,BR,Brasil,AC,Acre,1200138,Bujari,10266,Acre,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-04-26,1.0,0.0,AC,0.0,0.0,1.0,0.0,2020-04-27 18:41:03
4,BR,Brasil,AC,Acre,1200179,Capixaba,11733,Acre,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-27 18:41:03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,BR,Brasil,TO,Tocantins,1721208,Tocantinópolis,22870,Região de Saúde Portal do Bico,2020-02-01,SUS,0.6,2020-02-01,SUS,8.0,2020-02-01,SUS,NaN,2020-04-26,1.0,0.0,TO,0.0,1.0,1.0,1.0,2020-04-27 18:41:03
5566,BR,Brasil,TO,Tocantins,1721257,Tupirama,1891,Região de Saúde Cultura do Cerrado,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-27 18:41:03
5567,BR,Brasil,TO,Tocantins,1721307,Tupiratins,2671,Região de Saúde de Araguaia-Tocantins,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-27 18:41:03
5568,BR,Brasil,TO,Tocantins,1722081,Wanderlândia,11683,Região de Saúde Médio Norte,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-27 18:41:03


## Calculate dday

In [27]:
def calculate_recovered(df, pop_params):
    """
    Calcula recuperados ajustando o total de casos com a taxa de notificação.
    """

    confirmed_adjusted = df['confirmed_cases'].sum()/df['notification_rate']
    pop_params['R'] = confirmed_adjusted - pop_params['I'] - pop_params['D']
    pop_params['R'] = pop_params['R'].fillna(0).values[0]

    if pop_params['R'] < 0:
        pop_params['R'] = confirmed_adjusted - pop_params['D']
        pop_params['R'] = pop_params['R'].fillna(0).values[0]

    return pop_params

def dday_all(city_id, supply_type='n_beds'):
    """
    Calcula dday de leitos para a cidade em cityid.
    
    supply_type (str): n_beds / n_ventilators
    """
    
    if np.isnan(city_id) == True:
        return (None, None)
    
    # Dataframe com dados historicos de casos da cidade
    df_city = data[data['city_id'] == city_id]

    if df_city['infectious_period_cases'].values[0] < 0:
        return (None, None)
    
    params = dict()
    # Parametros populacionais
    params['population'] = {
        'N': df_city['population'].values[0],
        'I': df_city['active_cases'].fillna(1).values[0],
        'D': df_city['deaths'].fillna(0).values[0]
    }
    
    # Calcula recuperados
    params['population'] = calculate_recovered(df_city, params['population'])
    
    # Parametros de capacidade hospitalar
    params['supply'] = {
        'n_beds': df_city['number_beds'].values[0],
        'n_ventilators': df_city['number_ventilators'].values[0]
    }
    
    # Parâmetros da simulação: abertura == cenário nothing por 90 dias
    params['strategy'] = {
            'isolation': 90,
            'lockdown': 90
        }
    
    # Calcula dday de [supply_type] para o municipio
    try:
        dfs = simulator.run_simulation(params['population'], params['strategy'], config)
    except Exception as e:
        print('\n\n', city_id)
        print(e)
        
    ddays = simulator.get_dday(dfs, 'I2', params['supply'][supply_type])
    return ddays['best'], ddays['worst']

In [28]:
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

In [29]:
dday_cities = pd.DataFrame({'city_id': data['city_id'].drop_duplicates()})

In [30]:
dday_cities['dday_ventilators_best'], dday_cities['dday_ventilators_worst'] = zip(*dday_cities['city_id'].progress_apply(dday_all, supply_type='n_ventilators'))

In [31]:
dday_cities['dday_beds_best'], dday_cities['dday_beds_worst'] = zip(*dday_cities['city_id'].progress_apply(dday_all, supply_type='n_beds'))

In [32]:
dday_cities

,city_id,dday_ventilators_best,dday_ventilators_worst,dday_beds_best,dday_beds_worst
0,1200013,-1,-1,23,22
1,1200054,-1,-1,26,25
2,1200104,-1,-1,33,32
3,1200138,-1,-1,-1,-1
4,1200179,-1,-1,-1,-1
...,...,...,...,...,...
5565,1721208,17,16,33,32
5566,1721257,-1,-1,-1,-1
5567,1721307,-1,-1,-1,-1
5568,1722081,-1,-1,-1,-1


In [33]:
ddays_full = data.merge(dday_cities, on='city_id')
ddays_full

,country_iso,country_name,state_id,state_name,city_id,city_name,population,health_system_region,last_updated_number_ventilators,author_number_ventilators,number_ventilators,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_icu_beds,author_number_icu_beds,number_icu_beds,last_updated,confirmed_cases,deaths,state,daily_cases,infectious_period_cases,notification_rate,active_cases,data_last_refreshed,dday_ventilators_best,dday_ventilators_worst,dday_beds_best,dday_beds_worst
0,BR,Brasil,AC,Acre,1200013,Acrelândia,15256,Acre,2020-02-01,SUS,NaN,2020-02-01,SUS,3.2,2020-02-01,SUS,NaN,2020-04-26,12.0,0.0,AC,0.0,2.0,1.0,2.0,2020-04-27 18:41:03,-1,-1,23,22
1,BR,Brasil,AC,Acre,1200054,Assis Brasil,7417,Brasiléia,2020-02-01,SUS,NaN,2020-02-01,SUS,2.4,2020-02-01,SUS,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-27 18:41:03,-1,-1,26,25
2,BR,Brasil,AC,Acre,1200104,Brasiléia,26278,Brasiléia,2020-02-01,SUS,NaN,2020-02-01,SUS,8.0,2020-02-01,SUS,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-27 18:41:03,-1,-1,33,32
3,BR,Brasil,AC,Acre,1200138,Bujari,10266,Acre,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-04-26,1.0,0.0,AC,0.0,0.0,1.0,0.0,2020-04-27 18:41:03,-1,-1,-1,-1
4,BR,Brasil,AC,Acre,1200179,Capixaba,11733,Acre,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-27 18:41:03,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,BR,Brasil,TO,Tocantins,1721208,Tocantinópolis,22870,Região de Saúde Portal do Bico,2020-02-01,SUS,0.6,2020-02-01,SUS,8.0,2020-02-01,SUS,NaN,2020-04-26,1.0,0.0,TO,0.0,1.0,1.0,1.0,2020-04-27 18:41:03,17,16,33,32
5566,BR,Brasil,TO,Tocantins,1721257,Tupirama,1891,Região de Saúde Cultura do Cerrado,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-27 18:41:03,-1,-1,-1,-1
5567,BR,Brasil,TO,Tocantins,1721307,Tupiratins,2671,Região de Saúde de Araguaia-Tocantins,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-27 18:41:03,-1,-1,-1,-1
5568,BR,Brasil,TO,Tocantins,1722081,Wanderlândia,11683,Região de Saúde Médio Norte,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-27 18:41:03,-1,-1,-1,-1


In [34]:
ddays_full[ddays_full['city_name'] == 'Manaus']

,country_iso,country_name,state_id,state_name,city_id,city_name,population,health_system_region,last_updated_number_ventilators,author_number_ventilators,number_ventilators,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_icu_beds,author_number_icu_beds,number_icu_beds,last_updated,confirmed_cases,deaths,state,daily_cases,infectious_period_cases,notification_rate,active_cases,data_last_refreshed,dday_ventilators_best,dday_ventilators_worst,dday_beds_best,dday_beds_worst
161,BR,Brasil,AM,Amazonas,1302603,Manaus,2182763,Entorno de Manaus,2020-02-01,SUS,168.8,2020-02-01,SUS,674.4,2020-02-01,SUS,NaN,2020-04-26,2722.0,246.0,AM,44.0,1669.0,0.229227,7280.985442,2020-04-27 18:41:03,1,1,1,1


In [36]:
ddays_full[ddays_full['city_name'] == 'Rio de Janeiro']

,country_iso,country_name,state_id,state_name,city_id,city_name,population,health_system_region,last_updated_number_ventilators,author_number_ventilators,number_ventilators,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_icu_beds,author_number_icu_beds,number_icu_beds,last_updated,confirmed_cases,deaths,state,daily_cases,infectious_period_cases,notification_rate,active_cases,data_last_refreshed,dday_ventilators_best,dday_ventilators_worst,dday_beds_best,dday_beds_worst
3660,BR,Brasil,RJ,Rio de Janeiro,3304557,Rio de Janeiro,6718903,Metropolitana I,2020-02-01,SUS,865.4,2020-02-01,SUS,2786.6,2020-02-01,SUS,NaN,2020-04-26,4498.0,382.0,RJ,17.0,2502.0,0.244686,10225.34882,2020-04-27 18:41:03,1,1,12,12


In [37]:
ddays_full[ddays_full['city_name'] == 'São Paulo']

,country_iso,country_name,state_id,state_name,city_id,city_name,population,health_system_region,last_updated_number_ventilators,author_number_ventilators,number_ventilators,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_icu_beds,author_number_icu_beds,number_icu_beds,last_updated,confirmed_cases,deaths,state,daily_cases,infectious_period_cases,notification_rate,active_cases,data_last_refreshed,dday_ventilators_best,dday_ventilators_worst,dday_beds_best,dday_beds_worst
5348,BR,Brasil,SP,São Paulo,3550308,São Paulo,12252023,Grande São Paulo,2020-02-01,SUS,1438.6,2020-02-01,SUS,5569.4,2020-02-01,SUS,NaN,2020-04-26,13513.0,1114.0,SP,415.0,7161.0,0.25498,28084.598128,2020-04-27 18:41:03,1,1,10,9


In [40]:
ddays_full.to_csv('data/output/cities_capacity_overload_20200427.csv')

### Errors

- Cities with negative active cases cannot go over the calculation --> is it a problem with the data or code?

In [41]:
data[data['city_id'] == 4215604]['infectious_period_cases'].values[0] < 0

True

In [42]:
data[data['infectious_period_cases'] < 0]

,country_iso,country_name,state_id,state_name,city_id,city_name,population,health_system_region,last_updated_number_ventilators,author_number_ventilators,number_ventilators,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_icu_beds,author_number_icu_beds,number_icu_beds,last_updated,confirmed_cases,deaths,state,daily_cases,infectious_period_cases,notification_rate,active_cases,data_last_refreshed
657,BR,Brasil,CE,Ceará,2303303,Cariús,18699,Iguatu,2020-02-01,SUS,NaN,2020-02-01,SUS,3.2,2020-02-01,SUS,NaN,2020-04-26,1.0,1.0,CE,0.0,-1.0,0.02,-50.0,2020-04-27 18:41:03
1066,BR,Brasil,GO,Goiás,5217302,Pirenópolis,24908,Pirineus,2020-02-01,SUS,1.6,2020-02-01,SUS,10.2,2020-02-01,SUS,NaN,2020-04-26,0.0,0.0,GO,0.0,-1.0,NaN,NaN,2020-04-27 18:41:03
1119,BR,Brasil,GO,Goiás,5221601,Uruaçu,40532,Serra da Mesa,2020-02-01,SUS,1.8,2020-02-01,SUS,31.2,2020-02-01,SUS,NaN,2020-04-26,0.0,0.0,GO,0.0,-1.0,NaN,NaN,2020-04-27 18:41:03
2047,BR,Brasil,MG,Minas Gerais,3160405,Santo Antônio do Monte,28243,Divinópolis,2020-02-01,SUS,1.2,2020-02-01,SUS,10.0,2020-02-01,SUS,NaN,2020-04-26,3.0,0.0,MG,0.0,-1.0,1.00,-1.0,2020-04-27 18:41:03
4646,BR,Brasil,SC,Santa Catarina,4215604,Santa Rosa de Lima,2142,Tubarão,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-02-01,SUS,NaN,2020-04-26,0.0,0.0,SC,0.0,-2.0,NaN,NaN,2020-04-27 18:41:03
